In [1]:
! pip install semantic-kernel -U 
! pip install qdrant_client -U 
! pip install pyautogen -U

In [2]:
import uuid

In [3]:
import autogen
import semantic_kernel as sk
from dotenv import load_dotenv
from autogen.cache import Cache
import semantic_kernel.connectors.ai.open_ai as skaoai
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [4]:
def saveblog(blog):
    with open('./blogs/blog-'+ str(uuid.uuid4()) +'.md', 'w') as f:
        f.write(blog)
    return "saved"

In [5]:
async def writeblog(content):
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
    base_plugin = "./plugins/"
    write_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "WritePlugin")
    blogFunc = write_plugin["Blogs"]
    result = await blogFunc(content)
    return result.result

In [6]:
# @user_proxy.register_for_execution()
async def ask(question):
    kernel = sk.Kernel()
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


    kernel.add_text_embedding_generation_service(
            "embeddings_services", skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
#     kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))
#     kernel.add_text_embedding_generation_service(
#             "embeddings_services", skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview")
#     )
    base_plugin = "./plugins"
    answer_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "AnswerPlugin")
    qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
    kernel.register_memory_store(memory_store=qdrant_store)

    ask = question
    base_vectordb = "aboutMLKBDemoDemo"


    memories = await kernel.memory.search(
        base_vectordb, ask, limit=1, min_relevance_score=0.8
    )

    result = ''
    for memory in memories:
        print(f"Top Result: {memory.text} with score {memory.relevance}")
        result = memory.text

    answerFunc = answer_plugin["Summary"]

    summary_result = await answerFunc(result)

    return summary_result.result

In [7]:
config_list = autogen.config_list_from_json(
    "AOAI_CONFIG_LIST",
    filter_dict={
        "model": ["GPT4Model","GPT3Model"],
    },
)

In [8]:

llm_config={
    "config_list": config_list,
    "functions": [
        {
            "name": "ask",
            "description": "ask question about Machine Learning,  get basic knowledge",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "About Machine Learning",
                    }
                },
                "required": ["question"],
            },
        },
        {
            "name": "writeblog",
            "description": "write blogs in markdown format",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "basic content",
                    }
                },
                "required": ["content"],
            },
        },
        {
            "name": "saveblog",
            "description": "save blogs",
            "parameters": {
                "type": "object",
                "properties": {
                    "blog": {
                        "type": "string",
                        "description": "basic content",
                    }
                },
                "required": ["blog"],
            },
        }
    ],
}

In [9]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

user_proxy.register_function(
    function_map={
        "ask": ask,
        "writeblog": writeblog,
        "saveblog": saveblog
    }
)

In [10]:

with Cache.disk():
    await user_proxy.a_initiate_chat(
        assistant,
        message="""
            I'm writing a blog about Machine Learning. Find the answers to the 3 questions below and write an introduction based on them. After preparing these basic materials, write a blog and save it.

            1. What is Machine Learning?
            2. The difference between AI and ML
            3. The history of Machine Learning

            Let's go
    """
    )

user_proxy (to assistant):


            I'm writing a blog about Machine Learning. Find the answers to the 3 questions below and write an introduction based on them. After preparing these basic materials, write a blog and save it.

            1. What is Machine Learning?
            2. The difference between AI and ML
            3. The history of Machine Learning

            Let's go
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: ask *****
Arguments: 

            {
                "question": "What is Machine Learning?"
            }
****************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION ask...
Top Result: Introduction to machine learning - This course introduces the basics of machine learning (ML), a subset of artificial intelligence (AI) that uses algorithms to find hidden pattern